**Mounted Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**get_data**

In [2]:
import os

data_link = "/content/drive/MyDrive/largeProject/code/NER/dataset/"
list_link = os.listdir(data_link)

In [3]:
import json
from tqdm import tqdm

datas = []

for link in tqdm(list_link):
  with open(data_link + link) as f:
    data = json.loads(f.read())
  datas.append(data)

100%|██████████| 12/12 [00:13<00:00,  1.12s/it]


In [4]:
import random

datas[0][105]

{'sentence': 'Liệu chỉ tiêu Nhóm dịch vụ viễn thông di động này có kết quả không đạt từ giữa n2021 bắt đầu đến hết thg1 2021 ',
 'ner': {'<tên chỉ tiêu>': [[14, 45, 'Nhóm dịch vụ viễn thông di động']],
  '<năm>': [[80, 84, '2021'], [106, 110, '2021']],
  '<tháng>': [[104, 105, '1']]}}

In [5]:
import string

punctuation = string.punctuation
print(punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [6]:
from copy import deepcopy

def ner_convert(ner_dict):
    list_ner = []
    for k,v_list in ner_dict.items():
      for v in v_list:
         v_new = deepcopy(v)
         v_new.append(k)
         list_ner.append(v_new)
    list_ner = sorted(list_ner, key=lambda item: item[0])
    return list_ner

In [7]:
ner_convert(datas[0][4]['ner'])

[[14, 15, '3', '<tháng>'],
 [16, 20, '2020', '<năm>'],
 [30,
  90,
  'Xử lý PAKH dịch vụ Kênh truyền - Xử lý PAKH trong hạn 24 giờ',
  '<tên chỉ tiêu>']]

In [9]:
datas[0][49]

{'sentence': 'Từ đầu năm 2022 tính đến th 4 2022 t có một vấn đề cần được giải đáp chỉ tiêu Tổng đài 18009000 đã đem lại không đạt ',
 'ner': {'<năm>': [[11, 15, '2022'], [30, 34, '2022']],
  '<tháng>': [[28, 29, '4']],
  '<tên chỉ tiêu>': [[78, 95, 'Tổng đài 18009000']]}}

**Processing_data**

In [10]:
def generate_ner(data):
    flag_end = -1
    sentence = data['sentence']
    ner = data['ner']
    list_iter = ner_convert(ner)

    list_ner = []
    list_sentence = []
    max_len = len(list_ner)
    for i,k in enumerate(list_iter):
        v = list_iter[i]
        if i == 0:
            end = v[0]
            sent = sentence[0:end].strip().split(' ')
            for word in sent:
                if word != '':
                    list_ner.append('O')
                    list_sentence.append(word)
        if flag_end == -1:
           flag_end = v[1]
        else:
           sent = sentence[flag_end:v[0]].strip().split(' ')
           for word in sent:
              if word != '':
                list_ner.append('O')
                list_sentence.append(word)
           flag_end = v[1]
        #ner
        sent = sentence[v[0]:v[1]].strip().split(' ')
        for index,word in enumerate(sent):
            if index == 0:
                list_ner.append(f'B_{v[3]}')
            else:
                list_ner.append(f'I_{v[3]}')
            list_sentence.append(word)

        if i == max_len - 1:
            begin = v[1]
            sent = sentence[begin:].strip().split(' ')
            for word in sent:
                if word != '':
                    list_ner.append('O')
                    list_sentence.append(word)

    return list_ner, list_sentence

In [11]:
a,b = generate_ner(datas[0][49])

In [12]:
for c,d in zip(a,b):
    print(c,d)

O Từ
O đầu
O năm
B_<năm> 2022
O tính
O đến
O th
B_<tháng> 4
B_<năm> 2022
O t
O có
O một
O vấn
O đề
O cần
O được
O giải
O đáp
O chỉ
O tiêu
B_<tên chỉ tiêu> Tổng
I_<tên chỉ tiêu> đài
I_<tên chỉ tiêu> 18009000


In [13]:
import json
from tqdm import tqdm

ner_tags = set()
list_ners = []
list_sents = []

for link in list_link:
  with open(data_link + link) as f:
    data = json.loads(f.read())
  for ele in tqdm(data):
    list_ner, list_sentence = generate_ner(ele)
    list_ners.append(list_ner)
    list_sents.append(list_sentence)

    for tag in list_ner:
      ner_tags.add(tag)

100%|██████████| 10000/10000 [00:00<00:00, 34610.84it/s]


In [14]:
len(ner_tags)

13

In [15]:
l2i = {ele : i for i,ele in enumerate(ner_tags)}
i2l = {i : ele for i,ele in enumerate(ner_tags)}

In [16]:
l2i

{'I_<tên cụm chỉ tiêu>': 0,
 'I_<đơn vị>': 1,
 'B_<tên tổng công ty>': 2,
 'I_<tên chỉ tiêu>': 3,
 'B_<quý>': 4,
 'B_<tên chỉ tiêu>': 5,
 'O': 6,
 'B_<năm>': 7,
 'B_<đơn vị>': 8,
 'B_<năm m>': 9,
 'B_<tháng>': 10,
 'B_<tháng 1>': 11,
 'B_<tên cụm chỉ tiêu>': 12}

**Word segment**

In [17]:
!pip install py_vncorenlp

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.4 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4306 sha256=b46c7385d2fc36890ca5cd8c56eebef1f13eb09ac6b753f15b73321da5cbcd04
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [18]:
import py_vncorenlp

py_vncorenlp.download_model(save_dir='/content/')

In [19]:
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/content/')

In [20]:
list_ners[2]

['O',
 'B_<tháng>',
 'B_<năm>',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B_<tên chỉ tiêu>',
 'I_<tên chỉ tiêu>',
 'I_<tên chỉ tiêu>',
 'I_<tên chỉ tiêu>',
 'I_<tên chỉ tiêu>',
 'I_<tên chỉ tiêu>',
 'I_<tên chỉ tiêu>',
 'I_<tên chỉ tiêu>',
 'I_<tên chỉ tiêu>',
 'I_<tên chỉ tiêu>',
 'I_<tên chỉ tiêu>',
 'I_<tên chỉ tiêu>',
 'I_<tên chỉ tiêu>']

In [21]:
list_sents[2]

['Tháng',
 '2',
 '2020',
 'có',
 'thể',
 'cho',
 'tui',
 'biết',
 'chỉ',
 'tiêu',
 'Tỷ',
 'lệ',
 'xử',
 'lý',
 'trong',
 'hạn',
 '(Kênh',
 'BO)',
 '-',
 'Dịch',
 'vụ',
 'Chuyển',
 'tiền']

In [22]:
def test_ner(ner,sent):
    try:
      assert(len(sent) == len(ner))
      token_test = rdrsegmenter.word_segment(" ".join(sent))[0]

      token_test = token_test.replace("( ","(").replace(" )",")") \
                              .replace(" / ","/").replace("/ ","/").replace(" /","/") \
                              .replace("24 h","24h").replace("4 h","4h").replace("2 h","2h") \
                              .replace(" ,",",").replace(" ?","?").replace("10 K","10K") \
                              .replace("1 TTB","1TTB").split(' ')
      idx = 0
      ner_test = []
      # print(len(ner),"=======")
      # print(len(sent),"=======")
      # print(index)
      for token in token_test:
          # print(idx,"<<<<<<<<<<<<",token)
          ner_test.append(ner[idx])
          idx += len(token.split('_'))
      print(ner_test,token_test)
      assert(len(token_test) == len(ner_test))

      return ner_test, token_test
    except:
      print("=============")
      print(index)
      print("=============")

In [23]:
a,b = test_ner(list_ners[2],list_sents[2])

for c,d in zip(a,b):
    print(c,d)

['O', 'B_<tháng>', 'B_<năm>', 'O', 'O', 'O', 'O', 'O', 'B_<tên chỉ tiêu>', 'I_<tên chỉ tiêu>', 'I_<tên chỉ tiêu>', 'I_<tên chỉ tiêu>', 'I_<tên chỉ tiêu>', 'I_<tên chỉ tiêu>', 'I_<tên chỉ tiêu>', 'I_<tên chỉ tiêu>', 'I_<tên chỉ tiêu>', 'I_<tên chỉ tiêu>'] ['Tháng', '2', '2020', 'có_thể', 'cho', 'tui', 'biết', 'chỉ_tiêu', 'Tỷ_lệ', 'xử_lý', 'trong', 'hạn', '(Kênh', 'BO)', '-', 'Dịch_vụ', 'Chuyển', 'tiền']
O Tháng
B_<tháng> 2
B_<năm> 2020
O có_thể
O cho
O tui
O biết
O chỉ_tiêu
B_<tên chỉ tiêu> Tỷ_lệ
I_<tên chỉ tiêu> xử_lý
I_<tên chỉ tiêu> trong
I_<tên chỉ tiêu> hạn
I_<tên chỉ tiêu> (Kênh
I_<tên chỉ tiêu> BO)
I_<tên chỉ tiêu> -
I_<tên chỉ tiêu> Dịch_vụ
I_<tên chỉ tiêu> Chuyển
I_<tên chỉ tiêu> tiền


In [ ]:
k

In [ ]:
list_sents[2]

['Th',
 '12',
 '2020',
 'chi',
 'tiết',
 'hơn',
 'của',
 'cụm',
 'chỉ',
 'tiêu',
 'Tỷ',
 'lệ',
 'phản',
 'ánh',
 'với',
 'những',
 'chỉ',
 'tiêu',
 'đã',
 'được',
 'đánh',
 'giá',
 'là',
 'hoàn',
 'thành',
 'liên',
 'quan',
 'VTT']

In [ ]:
list_ners[2]

['O',
 'B_<tháng>',
 'B_<năm>',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B_<tên cụm chỉ tiêu>',
 'I_<tên cụm chỉ tiêu>',
 'I_<tên cụm chỉ tiêu>',
 'I_<tên cụm chỉ tiêu>',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B_<tên tổng công ty>']

In [24]:
new_sents = []
new_ners = []
index = 0

for i,ner in tqdm(enumerate(list_ners)):
    sent = list_sents[i]
    try:
      assert(len(sent) == len(ner))
      token_test = rdrsegmenter.word_segment(" ".join(sent))[0]

      token_test = token_test.replace("( ","(").replace(" )",")") \
                              .replace(" / ","/").replace("/ ","/").replace(" /","/") \
                              .replace("24 h","24h").replace("4 h","4h").replace("2 h","2h") \
                              .replace(" ,",",").replace(" ?","?").replace("10 K","10K") \
                              .replace("1 TTB","1TTB").split(' ')
      idx = 0
      ner_test = []
      # print(len(ner),"=======")
      # print(len(sent),"=======")
      # print(index)
      for token in token_test:
          # print(idx,"<<<<<<<<<<<<",token)
          ner_test.append(ner[idx])
          idx += len(token.split('_'))
      new_sents.append(token_test)
      new_ners.append(ner_test)

      index += 1
      assert(len(token_test) == len(ner_test))
    except:
      print("=============")
      print(index)
      print("=============")
      break

120000it [02:10, 917.10it/s] 


In [25]:
i2l

{0: 'I_<tên cụm chỉ tiêu>',
 1: 'I_<đơn vị>',
 2: 'B_<tên tổng công ty>',
 3: 'I_<tên chỉ tiêu>',
 4: 'B_<quý>',
 5: 'B_<tên chỉ tiêu>',
 6: 'O',
 7: 'B_<năm>',
 8: 'B_<đơn vị>',
 9: 'B_<năm m>',
 10: 'B_<tháng>',
 11: 'B_<tháng 1>',
 12: 'B_<tên cụm chỉ tiêu>'}

In [26]:
l2i

{'I_<tên cụm chỉ tiêu>': 0,
 'I_<đơn vị>': 1,
 'B_<tên tổng công ty>': 2,
 'I_<tên chỉ tiêu>': 3,
 'B_<quý>': 4,
 'B_<tên chỉ tiêu>': 5,
 'O': 6,
 'B_<năm>': 7,
 'B_<đơn vị>': 8,
 'B_<năm m>': 9,
 'B_<tháng>': 10,
 'B_<tháng 1>': 11,
 'B_<tên cụm chỉ tiêu>': 12}

In [ ]:
test_sentence = list_sents[60272]
test_ner = list_ners[60272]

In [ ]:
" ".join(test_sentence)

'Hãy giúp t hiểu hơn về được quản lý bởi tổng công ty VTT 1 2023 Phản ánh về CLM vô tuyến - Phản ánh về chất lượng sóng và thoại PA/1TTB/ngày nhận được là?'

In [ ]:
" ".join(rdrsegmenter.word_segment(" ".join(test_sentence))[0].replace("( ","(").replace(" )",")") \
                            .replace(" / ","/").replace("/ ","/").replace(" /","/") \
                            .replace("24 h","24h").split(' '))

'Hãy giúp t hiểu hơn về được quản_lý bởi tổng_công_ty VTT 1 2023 Phản_ánh về CLM vô_tuyến - Phản_ánh về chất_lượng sóng và thoại PA/1 TTB/ngày nhận được là ?'

In [27]:
import pickle

with open("/content/drive/MyDrive/largeProject/code/NER/l2i_new.pkl",'wb') as f:
  pickle.dump(l2i,f)

with open("/content/drive/MyDrive/largeProject/code/NER/i2l_new.pkl",'wb') as f:
  pickle.dump(i2l,f)

In [28]:
with open('/content/drive/MyDrive/largeProject/code/NER/l2i_new.pkl','rb') as f:
  data = pickle.load(f)

In [29]:
data

{'I_<tên cụm chỉ tiêu>': 0,
 'I_<đơn vị>': 1,
 'B_<tên tổng công ty>': 2,
 'I_<tên chỉ tiêu>': 3,
 'B_<quý>': 4,
 'B_<tên chỉ tiêu>': 5,
 'O': 6,
 'B_<năm>': 7,
 'B_<đơn vị>': 8,
 'B_<năm m>': 9,
 'B_<tháng>': 10,
 'B_<tháng 1>': 11,
 'B_<tên cụm chỉ tiêu>': 12}

In [30]:
new_list_ners = []

for list_ner in tqdm(new_ners):
  new_list_ner = [l2i[ele] for ele in list_ner]

  new_list_ners.append(new_list_ner)

100%|██████████| 120000/120000 [00:01<00:00, 105381.28it/s]


**Concat dataframe**

In [31]:
import pandas as pd

df = pd.DataFrame({
    "tokens" : new_sents,
    "ner_tags" : new_list_ners
})

In [32]:
df.to_csv("/content/drive/MyDrive/largeProject/code/NER/id_ner_new.csv")

In [33]:
len(new_sents)

120000

**Dataset_dict**

In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "vinai/phobert-base"
batch_size = 16

In [ ]:
!pip install datasets transformers seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.5 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=4ce84f1949b84be259c41388711910dd10257425de168a7e9c562c2fa938f3a0
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
datasets = load_dataset("csv",data_files = "/content/drive/MyDrive/largeProject/code/NER/id_ner.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
len(datasets["train"])

12000

In [ ]:
datasets["train"][0]

{'Unnamed: 0': 0,
 'tokens': "['Có', 'thể', 'cho', 'tôi', 'biết', 'trong', 'quý', '4', '2020', '11', 'Kênh', 'FO', '?']",
 'ner_tags': '[31, 31, 31, 31, 31, 31, 21, 22, 10, 14, 13, 8, 31]'}

In [ ]:
# datasets = datasets.remove_columns("Unnamed: 0")

datasets_train_test = datasets["train"].train_test_split(test_size=500).shuffle()
datasets_train_validation = datasets_train_test["train"].train_test_split(test_size=500).shuffle()

datasets["train"] = datasets_train_validation["train"]
datasets["validation"] = datasets_train_validation["test"]
datasets["test"] = datasets_train_test["test"]

n_samples_train = len(datasets["train"])
n_samples_validation = len(datasets["validation"])
n_samples_test = len(datasets["test"])
n_samples_total = n_samples_train + n_samples_validation + n_samples_test

print(f"- Training set: {n_samples_train*100/n_samples_total:.2f}%")
print(f"- Validation set: {n_samples_validation*100/n_samples_total:.2f}%")
print(f"- Test set: {n_samples_test*100/n_samples_total:.2f}%")

- Training set: 91.67%
- Validation set: 4.17%
- Test set: 4.17%


**Preproces_data**

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
